<span style="left: 13.82%; top: 29.55%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif;" role="presentation" dir="ltr">3</span><span style="left: 14.79%; top: 29.55%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif; transform: scaleX(0.958348);" role="presentation" dir="ltr">. Consider the following tables:</span> <span style="left: 38.86%; top: 29.55%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif; transform: scaleX(0.960571);" role="presentation" dir="ltr">Products(ID, ProductName)</span><span style="left: 59.45%; top: 29.55%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif;" role="presentation" dir="ltr">,</span> <span style="left: 60.55%; top: 29.55%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif; transform: scaleX(1.01926);" role="presentation" dir="ltr">Prices(ProductID REF Products(ID),</span>  
<span style="left: 16.09%; top: 30.97%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif; transform: scaleX(1.03721);" role="presentation" dir="ltr">Currency REF Rates(Currency), Price)</span><span style="left: 45.6%; top: 30.97%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif;" role="presentation" dir="ltr">,</span> <span style="left: 46.81%; top: 30.97%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif; transform: scaleX(1.01727);" role="presentation" dir="ltr">Rates(Currency, PricePLN)</span><span style="left: 67.42%; top: 30.97%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif; transform: scaleX(1.00179);" role="presentation" dir="ltr">. Note that a product price</span>  
<span style="left: 16.09%; top: 32.39%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif; transform: scaleX(1.00714);" role="presentation" dir="ltr">may not be declared in all known currencies, but it always is declared in PLN as a reference.</span>  
<span style="left: 16.09%; top: 33.81%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif; transform: scaleX(0.962704);" role="presentation" dir="ltr">Prepare a batch script to update the</span> <span style="left: 43.91%; top: 33.81%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif; transform: scaleX(0.959799);" role="presentation" dir="ltr">Prices</span> <span style="left: 49.15%; top: 33.81%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif; transform: scaleX(0.922866);" role="presentation" dir="ltr">table based on</span> <span style="left: 60.47%; top: 33.81%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif; transform: scaleX(0.924839);" role="presentation" dir="ltr">Rates</span> <span style="left: 65.21%; top: 33.81%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif; transform: scaleX(0.982383);" role="presentation" dir="ltr">table. In case there is a row in</span>  
<span style="left: 16.09%; top: 35.23%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif; transform: scaleX(0.959799);" role="presentation" dir="ltr">Prices</span> <span style="left: 21.28%; top: 35.23%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif; transform: scaleX(0.932892);" role="presentation" dir="ltr">which references a currency that no longer exists in</span> <span style="left: 59.54%; top: 35.23%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif; transform: scaleX(0.924839);" role="presentation" dir="ltr">Rates</span><span style="left: 63.54%; top: 35.23%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif; transform: scaleX(0.929289);" role="presentation" dir="ltr">, the row should be removed.</span>  
<span style="left: 16.09%; top: 37.12%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif; transform: scaleX(0.930781);" role="presentation" dir="ltr">This task should be implemented using cursors.</span>  
<span style="left: 16.09%; top: 38.54%; font-size: calc(var(--scale-factor)*9.96px); font-family: sans-serif; transform: scaleX(1.00445);" role="presentation" dir="ltr">[3p]</span>

In [ ]:
DROP TABLE IF EXISTS Prices;
DROP TABLE IF EXISTS Products;
DROP TABLE IF EXISTS Rates;

CREATE TABLE Products(
        ID INT IDENTITY PRIMARY KEY,
        ProductName VARCHAR(20)
        );

CREATE TABLE Rates(
	Currency VARCHAR(20) PRIMARY KEY,
	PricePLN FLOAT
	);

CREATE TABLE Prices(
	ProductID INT FOREIGN KEY REFERENCES Products(ID),
	Currency VARCHAR(20) FOREIGN KEY REFERENCES Rates(Currency),
	Price FLOAT
	);                              



In [ ]:
INSERT INTO Products(ProductName)
	VALUES
		('pro1'),
        ('pro2'),
        ('pro3'),
        ('pro4'),
        ('pro5');

INSERT INTO Rates(Currency, PricePLN)
	VALUES
		('GBP',2.231),
		('CZK',8.23),
		('USD', 3.231),
		('PLN',1),
		('EUR',4.532);
	
INSERT INTO Prices(ProductID, Currency, Price)
	VALUES
		(1,'PLN',120),
		(2,'PLN',10),
		(3,'PLN',560),
		(4,'PLN',30),
		(5,'PLN',20),
		(2,'GBP',20),
		(4,'EUR',32),
		(1,'USD',3),
		(5,'YAN',123);


In [ ]:
DECLARE
	@pricePLN FLOAT,
	@currency VARCHAR(20),
	@price FLOAT,
	@productID INT,
	@exchange FLOAT;

DECLARE update_prices CURSOR DYNAMIC
FOR
	SELECT 
		ProductID,
		Currency,
		Price
	FROM [dbo].[Prices];

OPEN update_prices;

BEGIN;
	DROP TABLE IF EXISTS PricesPLN;
	SELECT
		ProductID,
		Currency,
		Price
	INTO 
		PricesPLN
	FROM
		[dbo].[Prices] 
	WHERE
		Currency = 'PLN';

	FETCH NEXT FROM update_prices INTO
		@productID,
		@currency,
		@price;

	WHILE @@FETCH_STATUS = 0
		BEGIN;
			SET @exchange = (SELECT PricePLN FROM [dbo].[Rates] WHERE @currency = Currency)
			IF @exchange IS NOT NULL
				BEGIN;
					SET @pricePLN = (SELECT TOP 1 Price FROM PricesPLN WHERE ProductID = @productID);
					UPDATE Prices SET Price = @exchange * @pricePLN WHERE CURRENT OF update_prices;

				END;

		FETCH NEXT FROM update_prices INTO
			@productID,
			@currency,
			@price;
		END;
END;
